In [ ]:
# 백엔드연동 + 데이터 적재 + API + AI 모듈 + 성능 평가 통합 코드입니다.
# 제가 임시로 작성한거라, 하이퍼 파라미터 조정을 하면 변동될 수 있습니다. (일요일까지 완성본 올릴게요.)
# 제가 성능평가 2차를 할 땐 10개(product테이블의 더미데이터 포함) 테이블을 사용할 것입니다.
# 그러나 실제 백엔드에 적재할 땐 api를 연동하기 때문에 9개의 테이블만 사용해야 해서 적재 후, 변환이 필요할 수 있어요!
# FLASK 사용했습니다.
# image_url은 amazon s3에 적재할 예정입니다.

In [ ]:
# JSON 파일 생성

In [ ]:
# JSON파일은 제가 다 변환해서 일요일까지 올려드리겠습니다!
# 만약 문제가 생기실 경우, 자체적으로 DataFrame을 json파일로 변경하면 됩니다.
# 혹시 몰라 코드 작성해드릴게요.
import pandas as pd

# DataFrame 생성
food_table = pd.DataFrame(food_data)

# JSON 파일로 저장
food_table.to_json('food_data.json', orient='records')

In [ ]:
# Flask 백엔드 설정 및 데이터베이스 초기화

In [ ]:
from flask import Flask, jsonify, request, send_file
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import requests
import random
from collections import deque
import matplotlib.pyplot as plt
from werkzeug.utils import secure_filename
import boto3

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///data.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

In [ ]:
# S3 설정
# 아직 S3 안만들어서, 기본 키 사용했습니다.
S3_BUCKET = "your-s3-bucket-name" //채우는 것
S3_KEY = "your-aws-access-key-id" //채우는 것
S3_SECRET = "your-aws-secret-access-key" //채우는 것
S3_LOCATION = f"http://{S3_BUCKET}.s3.amazonaws.com/" //채우는 것

s3 = boto3.client(
    "s3",
    aws_access_key_id=S3_KEY,
    aws_secret_access_key=S3_SECRET
)

def upload_file_to_s3(file, bucket_name=S3_BUCKET, acl="public-read"):
    try:
        s3.upload_fileobj(
            file,
            bucket_name,
            file.filename,
            ExtraArgs={
                "ACL": acl,
                "ContentType": file.content_type
            }
        )
    except Exception as e:
        print("Something Happened: ", e)
        return e

    return f"{S3_LOCATION}{file.filename}"

def download_file_from_s3(filename, bucket_name=S3_BUCKET):
    try:
        s3.download_file(bucket_name, filename, filename)
        return filename
    except Exception as e:
        print("Something Happened: ", e)
        return None

In [ ]:
# 데이터베이스 모델 정의

In [ ]:
# 데이터베이스 모델 정의
class Allergy(db.Model):
    allergy_id = db.Column(db.Integer, primary_key=True)
    allergy_name = db.Column(db.String(50))

class Food(db.Model):
    food_id = db.Column(db.Integer, primary_key=True)
    food_name = db.Column(db.String(50))
    storage_method = db.Column(db.String(50))
    allergy_id = db.Column(db.Integer, db.ForeignKey('allergy.allergy_id'))
    is_vegan = db.Column(db.Boolean)
    food_image_url = db.Column(db.String(200))

    def as_dict(self):
        return {c.name: getattr(self, c.name) for c in self.__table__.columns}

class FridgeContents(db.Model):
    member_id = db.Column(db.Integer, primary_key=True)
    food_id = db.Column(db.Integer, primary_key=True)
    quantity = db.Column(db.Integer)
    purchase_date = db.Column(db.String(50))
    expiry_date = db.Column(db.String(50))

class RecipeIngredients(db.Model):
    recipe_ingredients_id = db.Column(db.Integer, primary_key=True)
    recipe_id = db.Column(db.Integer, db.ForeignKey('recipe.recipe_id'))
    food_id = db.Column(db.Integer, db.ForeignKey('food.food_id'))
    quantity = db.Column(db.Integer)

class Recipe(db.Model):
    recipe_id = db.Column(db.Integer, primary_key=True)
    recipe_name = db.Column(db.String(50))
    cuisine_type = db.Column(db.String(50))
    description = db.Column(db.String(200))
    recipe_image_url = db.Column(db.String(200))
    is_vegan = db.Column(db.Boolean)

    def as_dict(self):
        return {c.name: getattr(self, c.name) for c in self.__table__.columns}

class ShoppingLists(db.Model):
    shopping_list_id = db.Column(db.Integer, primary_key=True)
    member_id = db.Column(db.Integer, db.ForeignKey('member.member_id'))
    food_id = db.Column(db.Integer, db.ForeignKey('food.food_id'))
    quantity = db.Column(db.Integer)

class MemberAllergies(db.Model):
    member_id = db.Column(db.Integer, db.ForeignKey('member.member_id'), primary_key=True)
    allergy_id = db.Column(db.Integer, db.ForeignKey('allergy.allergy_id'), primary_key=True)

class Member(db.Model):
    member_id = db.Column(db.Integer, primary_key=True)
    member_name = db.Column(db.String(50))
    email = db.Column(db.String(50))
    password = db.Column(db.String(50))
    is_family_head = db.Column(db.Boolean)
    gender = db.Column(db.String(10))
    age_group = db.Column(db.String(10))
    is_vegan = db.Column(db.Boolean)
    preferred_cuisine = db.Column(db.String(50))
    non_preferred_cuisine = db.Column(db.String(50))
    preferred_ingredient = db.Column(db.String(50))
    nonpreferred_ingredient = db.Column(db.String(50))

    def as_dict(self):
        return {c.name: getattr(self, c.name) for c in self.__table__.columns}

class MemberPreferences(db.Model):
    member_preference_id = db.Column(db.Integer, primary_key=True)
    member_id = db.Column(db.Integer, db.ForeignKey('member.member_id'))
    recipe_id = db.Column(db.Integer, db.ForeignKey('recipe.recipe_id'))
    preference_level = db.Column(db.Integer)
    liked = db.Column(db.Boolean)

In [ ]:
# 데이터베이스 초기화 및 데이터 적재
@app.before_first_request
@app.before_first_request
def create_tables():
    db.create_all()

    # CSV 파일 로드
    allergy_table = pd.read_csv('allergy_table.csv')
    food_table = pd.read_csv('food_table.csv')
    fridge_contents_table = pd.read_csv('fridge_contents_table.csv')
    recipe_ingredients_table = pd.read_csv('recipe_ingredients_table.csv')
    recipes_table = pd.read_csv('recipes_table.csv')
    shopping_lists_table = pd.read_csv('shopping_lists_table.csv')
    member_allergies_table = pd.read_csv('member_allergies_table.csv')
    member_data_table = pd.read_csv('member_data_table.csv')
    member_preferences_table = pd.read_csv('member_preferences_table.csv')
# 제가 최종 csv파일, json파일 다 올릴게요!

In [ ]:
 # 데이터베이스에 데이터 적재
 # 데이터베이스에 데이터 적재
    for _, row in allergy_table.iterrows():
        allergy = Allergy(**row)
        db.session.add(allergy)

    for _, row in food_table.iterrows():
        food = Food(**row)
        db.session.add(food)

    for _, row in fridge_contents_table.iterrows():
        fridge_content = FridgeContents(**row)
        db.session.add(fridge_content)

    for _, row in recipe_ingredients_table.iterrows():
        recipe_ingredient = RecipeIngredients(**row)
        db.session.add(recipe_ingredient)

    for _, row in recipes_table.iterrows():
        recipe = Recipe(**row)
        db.session.add(recipe)

    for _, row in shopping_lists_table.iterrows():
        shopping_list = ShoppingLists(**row)
        db.session.add(shopping_list)

    for _, row in member_allergies_table.iterrows():
        member_allergy = MemberAllergies(**row)
        db.session.add(member_allergy)

    for _, row in member_data_table.iterrows():
        member = Member(**row)
        db.session.add(member)

    for _, row in member_preferences_table.iterrows():
        member_preference = MemberPreferences(**row)
        db.session.add(member_preference)

    db.session.commit()

In [ ]:
# AI 모델 로드
def load_model():
    # 모델 구조 로드
    with open("model.json", "r") as json_file:
        loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)

    # 모델 가중치 로드
    loaded_model.load_weights("model.h5")

    # 모델 컴파일
    loaded_model.compile(loss='mse', optimizer='adam')
    return loaded_model

In [ ]:
# 모델 로드
model = load_model()

In [ ]:
# 강화학습 환경 정의
class FridgeEnv:
    def __init__(self, fridge_contents_table, food_table, member_data_table, member_id):
        self.fridge_contents = fridge_contents_table[fridge_contents_table['member_id'] == member_id]
        self.food_table = food_table
        self.member_data = member_data_table[member_data_table['member_id'] == member_id]
        self.state_size = len(self.fridge_contents)
        self.action_size = len(food_table)
        self.reset()

    def reset(self):
        self.state = self.fridge_contents['quantity'].values
        return self.state

    def step(self, action):
        food_id = self.food_table.iloc[action]['food_id']
        food_quantity = 1  # 기본 주문 수량

        if food_id in self.fridge_contents['food_id'].values:
            idx = self.fridge_contents[self.fridge_contents['food_id'] == food_id].index[0]
            self.fridge_contents.at[idx, 'quantity'] += food_quantity
        else:
            new_entry = {'member_id': self.member_data['member_id'].values[0], 'food_id': food_id, 'quantity': food_quantity}
            self.fridge_contents = self.fridge_contents.append(new_entry, ignore_index=True)

        reward = 1 if food_quantity > 0 else 0
        done = False
        next_state = self.fridge_contents['quantity'].values

        return next_state, reward, done, {}

    def get_state(self):
        return self.state

In [ ]:
# 사용자 ID 설정
member_id = 1

# 환경 초기화
env = FridgeEnv(fridge_contents_table, food_table, member_data_table, member_id)

In [ ]:
# DQN 에이전트 정의
class DQNAgent:
    def __init__(self, state_size, action_size, access_token):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.access_token = access_token

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

    def order_food(self, product_id, quantity):
        # 신세계몰 API 호출
        return add_to_cart(self.access_token, product_id, quantity)

In [ ]:
# API 호출 부분입니다.

In [ ]:
# API 기본 URL
BASE_URL = "https://api.shinsegaemall.com"

# API 호출 함수
def call_api(endpoint, method='GET', headers=None, data=None, params=None):
    url = f"{BASE_URL}{endpoint}"
    response = requests.request(method, url, headers=headers, json=data, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

In [ ]:
# 로그인 함수
def login(email, password):
    endpoint = "/members/login"
    data = {
        "email": email,
        "password": password
    }
    response = call_api(endpoint, method='POST', data=data)
    return response['code'], response['data']['accessToken']

In [ ]:
# 장바구니 조회 함수
def get_cart(access_token):
    endpoint = "/products/list"
    headers = {
        "accessToken": access_token
    }
    return call_api(endpoint, headers=headers)

In [ ]:
# 장바구니에 품목 추가 함수
def add_to_cart(access_token, product_id, quantity):
    endpoint = "/products/count"
    headers = {
        "accessToken": access_token
    }
    data = {
        "productId": product_id,
        "quantity": quantity
    }
    return call_api(endpoint, method='PATCH', headers=headers, data=data)

In [ ]:
# 추천 레시피 함수
def recommend_recipes(member_id, fridge_contents, recipes_table, recipe_ingredients_table):
    recommended_recipes = []
    for recipe_id in recipes_table['recipe_id'].unique():
        ingredients = recipe_ingredients_table[recipe_ingredients_table['recipe_id'] == recipe_id]
        if all(item in fridge_contents['food_id'].values for item in ingredients['food_id'].values):
            recommended_recipes.append(recipe_id)
    return recommended_recipes

In [ ]:
# 추천 시스템 평가 함수
def evaluate_recommender(agent, env, members, recipes, interactions, num_recipes=5):
    test_members = interactions['member_id'].unique()
    precisions, recalls, f1_scores = [], [], []

    for member_id in test_members:
        env.reset()
        state = env.get_state()
        recommended_recipes = agent.act(state)

        actual_items = set(interactions[interactions['member_id'] == member_id]['recipe_id'])
        recommended_items = set(recommend_recipes(member_id, env.fridge_contents, recipes, interactions))

        tp = len(actual_items & recommended_items)
        fp = len(recommended_items - actual_items)
        fn = len(actual_items - recommended_items)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    average_precision = sum(precisions) / len(precisions)
    average_recall = sum(recalls) / len(recalls)
    average_f1_score = sum(f1_scores) / len(f1_scores)

    print(f'Average Precision: {average_precision}')
    print(f'Average Recall: {average_recall}')
    print(f'Average F1-Score: {average_f1_score}')

    return average_f1_score

In [ ]:
# 사용자 데이터 로드

In [ ]:
members = member_data_table
recipes = recipes_table
interactions = member_preferences_table

In [ ]:
# 강화학습 모델 학습 및 평가
state_size = env.state_size
action_size = env.action_size
email = "your_email@example.com"
password = "your_password"
status_code, access_token = login(email, password)
if status_code != 200:
    raise ValueError("Failed to log in")
agent = DQNAgent(state_size, action_size, access_token)

In [ ]:
# DQN 모델 학습
def train_dqn(episodes=1000):
    batch_size = 32
    total_rewards = []

    for e in range(episodes):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        total_reward = 0

        for time in range(500):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            total_reward += reward
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print(f"episode: {e}/{episodes}, score: {time}, e: {agent.epsilon:.2}")
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

        total_rewards.append(total_reward)
        agent.save("dqn_model.h5")
    return total_rewards

In [ ]:
@app.route('/')
def home():
    return "Welcome to the API!"

@app.route('/foods', methods=['GET'])
def get_foods():
    foods = Food.query.all()
    return jsonify([food.as_dict() for food in foods])

@app.route('/recipes', methods=['GET'])
def get_recipes():
    recipes = Recipe.query.all()
    return jsonify([recipe.as_dict() for recipe in recipes])

@app.route('/members', methods=['GET'])
def get_members():
    members = Member.query.all()
    return jsonify([member.as_dict() for member in members])

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    # 예측을 위한 입력 데이터 생성
    input_data = np.array([data['input']])
    # 예측 수행
    predictions = model.predict(input_data)
    return jsonify(predictions.tolist())

if __name__ == '__main__':
    app.run(debug=True)